In [19]:
import urllib
import json
import pandas as pd

In [20]:
def getlatlng(addr):
  client_id = '345bnyc1cf'
  client_secret = 'otoFCXOpzjknUtnDPH6YfkyhYwCS2WNapc2fNg6Q'

  encText = urllib.parse.quote(addr)
  url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=' + encText

  request = urllib.request.Request(url)
  request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
  request.add_header('X-NCP-APIGW-API-KEY', client_secret)

  res = urllib.request.urlopen(request)
  rescode = res.getcode()
  if (rescode == 200):
      response_body = res.read()
      decRes = response_body.decode('utf-8')
      data = json.loads(decRes)

  else:
      print("Error Code:" + rescode)

  lng = data['addresses'][0]['x']
  lat = data['addresses'][0]['y']

  return(lat, lng)

In [25]:
kyochon = pd.read_csv('data/kyochon_jungeun.csv', encoding='utf-8')
# drop row 0~672
kyochon = kyochon.drop(range(673))
kyochon

,가맹점명,주소
673,송정(구미),경북 구미시 송정동 38-5 우방타운 206동2호
674,송정(부산),부산 해운대구 송정동 227-7
675,송정1호,광주 광산구 도산동 1143
676,송정지구,울산 북구 송정동 407
677,송천,전북 전주시 덕진구 송천동2가 179-116
...,...,...
1337,휘닉스봉평,강원 평창군 봉평면 무이리 709
1338,흑석1호,서울 동작구 흑석동 180-6
1339,흥덕,경기 용인시 기흥구 영덕동 994-3
1340,흥업,강원 원주시 흥업면 흥업리 585-8


In [26]:
kyochon['위도'] = 0
kyochon['경도'] = 0

In [29]:
# 위도, 경도 추출
for i in range(len(kyochon)):
  try:
    addr = kyochon.iloc[i]['주소']
    lat, lng = getlatlng(addr)
    kyochon.iloc[i, 2] = lat
    kyochon.iloc[i, 3] = lng
  except:
    kyochon.iloc[i, 2] = 0
    kyochon.iloc[i, 3] = 0

In [36]:
kyochon = kyochon[kyochon['위도'] != 0]
kyochon

,가맹점명,주소,위도,경도
673,송정(구미),경북 구미시 송정동 38-5 우방타운 206동2호,36.1211882,128.3482611
674,송정(부산),부산 해운대구 송정동 227-7,35.1822099,129.2039977
675,송정1호,광주 광산구 도산동 1143,35.1291944,126.7917364
676,송정지구,울산 북구 송정동 407,35.6014505,129.3629638
677,송천,전북 전주시 덕진구 송천동2가 179-116,35.8667998,127.1310456
...,...,...,...,...
1337,휘닉스봉평,강원 평창군 봉평면 무이리 709,37.5849677,128.3265124
1338,흑석1호,서울 동작구 흑석동 180-6,37.5087015,126.9616708
1339,흥덕,경기 용인시 기흥구 영덕동 994-3,37.2740737,127.0716406
1340,흥업,강원 원주시 흥업면 흥업리 585-8,37.3019658,127.9193977


In [38]:
# save kyochon as kyochon_back
kyochon.to_csv('data/kyochon_back.csv', encoding='utf-8', index=False)

In [40]:
# get kyochon_front
kyochon_front = pd.read_csv('data/kyochon_front.csv', encoding='utf-8')

In [41]:
# append
kyochon_front = kyochon_front.append(kyochon)
kyochon_front

,가맹점명,주소,위도,경도
0,가락2호,서울 송파구 가락동 75-7,37.497009,127.120028
1,가람마을,경기 파주시 와동동 1557-1,37.734457,126.755333
2,가산디지털,서울 금천구 가산동 371-6 가산비지니스센터 2층,37.4802,126.881068
3,가야,부산 부산진구 가야동 478,35.147715,129.030712
4,가양,서울 강서구 등촌동 75-1,37.561262,126.853977
...,...,...,...,...
1337,휘닉스봉평,강원 평창군 봉평면 무이리 709,37.5849677,128.3265124
1338,흑석1호,서울 동작구 흑석동 180-6,37.5087015,126.9616708
1339,흥덕,경기 용인시 기흥구 영덕동 994-3,37.2740737,127.0716406
1340,흥업,강원 원주시 흥업면 흥업리 585-8,37.3019658,127.9193977


In [42]:
import json
import folium
import urllib.request

In [49]:
map = folium.Map(
  location=[37.5502, 126.982], 
  zoom_start=7,
  tiles='Stamen Terrain',
)
# add marker
for i in range(len(kyochon_front)):
  lat = kyochon_front.iloc[i]['위도']
  lng = kyochon_front.iloc[i]['경도']
  name = kyochon_front.iloc[i]['가맹점명']
  addr = kyochon_front.iloc[i]['주소']
  folium.Marker(
    location=[lat, lng],
    popup=folium.Popup(
      '<div style="text-align: center;">' + name + '점<br>' + addr + '</div>',
      max_width=350,
    ),
    icon=folium.Icon(
      icon='pig',
      color='blue',
    ),
  ).add_to(map)

# save map
map